In [17]:
import json
import pandas as pd
import itertools

In [ ]:
wictionary_file_path = "kaikki.org-dictionary-English.jsonl"

with open(wictionary_file_path, "r", encoding="utf-8") as f:
    for i in range(5):
        line = f.readline().strip()
        #if line:
            #print(json.loads(line))

In [19]:
import json

# Define file paths
wictionary_file_path = "kaikki.org-dictionary-English.jsonl"  # Replace with your actual file path
output_file = "english_translations.jsonl"

# Process the file
with open(wictionary_file_path, "r", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
    for line in f_in:
        try:
            entry = json.loads(line.strip())  # Read JSON line
            english_word = entry.get("word", "")
            raw_glosses = []
            
            if "senses" not in entry:
                continue

            for sense in entry["senses"]:
                if "raw_glosses" not in sense or "translations" not in sense:
                    continue
                
                translations = []
                for translation in sense["translations"]:
                    lang = translation.get("lang", "unknown")
                    word = translation.get("word", "")
                    translations.append({"lang": lang, "word": word})
                    raw_glosses = sense["raw_glosses"]

                if not translations:
                    continue

                json_line = json.dumps(
                    {"word": english_word, 
                        "translations": translations,
                        "raw_glosses": raw_glosses
                    }, 
                    ensure_ascii=False)
                f_out.write(json_line + "\n")  # Write JSON line with newline
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {e}")

print(f"Success! Saved all words in {output_file}.")

Success! Saved all words in english_translations.jsonl.


In [20]:
english_translations = "english_translations.jsonl"
# Load the data
data = []
with open(english_translations, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))

print(f"Loaded {len(data)} entries.")

Loaded 81349 entries.


In [21]:
# First 5 entries
for i, entry in enumerate(data[:5]):  # Print the first 5 entries
    print(f"Entry {i + 1}: {entry}")

Entry 1: {'word': 'free', 'translations': [{'lang': 'Afrikaans', 'word': 'vrye'}, {'lang': 'Albanian', 'word': 'lirë (i/e)'}, {'lang': 'Amharic', 'word': 'ነፃ'}, {'lang': 'Arabic', 'word': 'حُرّ'}, {'lang': 'Arabic', 'word': 'حر'}, {'lang': 'Arabic', 'word': 'حر'}, {'lang': 'Armenian', 'word': 'արձակ'}, {'lang': 'Armenian', 'word': 'ազատ'}, {'lang': 'Aromanian', 'word': 'lefter'}, {'lang': 'Assamese', 'word': 'মুকলি'}, {'lang': 'Asturian', 'word': 'llibre'}, {'lang': 'Azerbaijani', 'word': 'azad'}, {'lang': 'Bambara', 'word': 'hɔrɔn'}, {'lang': 'Bashkir', 'word': 'ирекле'}, {'lang': 'Bashkir', 'word': 'азат'}, {'lang': 'Bashkir', 'word': 'бәйһеҙ'}, {'lang': 'Bashkir', 'word': 'хөр'}, {'lang': 'Belarusian', 'word': 'свабо́дны'}, {'lang': 'Belarusian', 'word': 'во́льны'}, {'lang': 'Bengali', 'word': 'মুক্ত'}, {'lang': 'Bengali', 'word': 'আজাদ'}, {'lang': 'Bikol Central', 'word': 'talingkas'}, {'lang': 'Bulgarian', 'word': 'свобо́ден'}, {'lang': 'Catalan', 'word': 'lliure'}, {'lang': 'Chin

In [22]:
rows = []
for entry in data:
    english_word = entry["word"]
    raw_glosses = entry["raw_glosses"]
    translations = entry["translations"]

    for translation in translations:
        lang = translation["lang"]
        word = translation["word"]
        rows.append({
            "english_word": english_word,
            "language": lang,
            "translation": word,
            "raw_glosses": raw_glosses
        })


In [ ]:
df = pd.DataFrame(rows)

In [24]:
df.head(10)

,english_word,language,translation,raw_glosses
0,free,Afrikaans,vrye,"[(social) Unconstrained., Not imprisoned or en..."
1,free,Albanian,lirë (i/e),"[(social) Unconstrained., Not imprisoned or en..."
2,free,Amharic,ነፃ,"[(social) Unconstrained., Not imprisoned or en..."
3,free,Arabic,حُرّ,"[(social) Unconstrained., Not imprisoned or en..."
4,free,Arabic,حر,"[(social) Unconstrained., Not imprisoned or en..."
5,free,Arabic,حر,"[(social) Unconstrained., Not imprisoned or en..."
6,free,Armenian,արձակ,"[(social) Unconstrained., Not imprisoned or en..."
7,free,Armenian,ազատ,"[(social) Unconstrained., Not imprisoned or en..."
8,free,Aromanian,lefter,"[(social) Unconstrained., Not imprisoned or en..."
9,free,Assamese,মুকলি,"[(social) Unconstrained., Not imprisoned or en..."


In [25]:
# Anzahl der Einträge pro Sprache
print(df["language"].value_counts())

# Anzahl der Einträge pro englischem Wort
print(df["english_word"].value_counts())

language
Finnish      70359
German       61777
Russian      54408
Spanish      51622
French       49372
             ...  
Dongolawi        1
Dompo            1
Domari           1
Dobu             1
Magyar           1
Name: count, Length: 4006, dtype: int64
english_word
water           4025
dog             1060
rain             975
order            927
fish             896
                ... 
neologize          1
oblique            1
cytomegaly         1
topspace           1
partitionist       1
Name: count, Length: 57231, dtype: int64


In [26]:
lang_long = df['language'].unique()
lang_long = pd.DataFrame(lang_long, columns=['language'])
lang_long

,language
0,Afrikaans
1,Albanian
2,Amharic
3,Arabic
4,Armenian
...,...
4001,Church Slavonic
4002,Shaozhou Tuhua
4003,Finish
4004,Ulster Scots


In [27]:
lang_long = lang_long[lang_long['language'].str.contains(',') | lang_long['language'].str.contains(r'\d')]

lang_long

for lang in lang_long["language"]:
    print(lang)

1895, Sir Arthur Conan Doyle, chapter 12, in The Stark Munro Letters


In [28]:
df['is_long'] = df["language"].apply(lambda x: x in list(lang_long["language"]))
cleaned = df[df["is_long"] == False]

print(df.shape[0], cleaned.shape[0])


1197714 1197713


In [29]:
cleaned.head(100)

,english_word,language,translation,raw_glosses,is_long
0,free,Afrikaans,vrye,"[(social) Unconstrained., Not imprisoned or en...",False
1,free,Albanian,lirë (i/e),"[(social) Unconstrained., Not imprisoned or en...",False
2,free,Amharic,ነፃ,"[(social) Unconstrained., Not imprisoned or en...",False
3,free,Arabic,حُرّ,"[(social) Unconstrained., Not imprisoned or en...",False
4,free,Arabic,حر,"[(social) Unconstrained., Not imprisoned or en...",False
...,...,...,...,...,...
95,free,Polish,swobodny,"[(social) Unconstrained., Not imprisoned or en...",False
96,free,Portuguese,livre,"[(social) Unconstrained., Not imprisoned or en...",False
97,free,Quechua,qispi,"[(social) Unconstrained., Not imprisoned or en...",False
98,free,Romagnol,lèbar,"[(social) Unconstrained., Not imprisoned or en...",False
